## Load the data 

In [1]:
import numpy as np
import pandas as pd

from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext

import sparknlp


In [2]:
data=pd.read_csv('./amazon_reviews_us_Baby_v1_00.tsv',on_bad_lines='skip',sep='\t',nrows=5000)

In [3]:
data.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,9970739,R8EWA1OFT84NX,B00GSP5D94,329991347,Summer Infant SwaddleMe Adjustable Infant Wrap...,Baby,5,0,0,N,Y,Great swaddled blankets,Loved these swaddle blankets and so did my dau...,2015-08-31
1,US,23538442,R2JWY4YRQD4FOP,B00YYDDZGU,646108902,Pacifier Clip Girl (3 Pack) Ziggy Baby 2-Sided...,Baby,5,0,0,N,N,Too cute and really nice,These are adorable pacifier clips. SavvyBaby h...,2015-08-31
2,US,8273344,RL5ESX231LZ0B,B00BUBNZC8,642922361,Udder Covers - Breast Feeding Nursing Cover,Baby,5,0,0,N,Y,Five Stars,Great gift,2015-08-31
3,US,24557753,RRMS9ZWJ2KD08,B00AWLZFTS,494272733,Gerber Graduates Fun Pack Utensils,Baby,5,0,0,N,Y,Cute; wash up nicely in dishwasher.,These forks are great for my 10 month old daug...,2015-08-31
4,US,46263340,R14I3ZG5E6S7YM,B00KM60D3Q,305813185,Summer Infant Ultra Sight Pan/Scan/Zoom Video ...,Baby,5,0,0,N,Y,Love it!,I wanted something for piece of mind with my l...,2015-08-31


In [4]:
data.describe()

,customer_id,product_parent,star_rating,helpful_votes,total_votes
count,5.000000e+03,5.000000e+03,5000.000000,5000.000000,5000.000000
mean,2.459909e+07,4.894207e+08,4.210200,0.641200,0.834200
std,1.572903e+07,2.849076e+08,1.282788,7.071921,7.367331
min,1.013400e+04,2.341800e+04,1.000000,0.000000,0.000000
25%,1.193007e+07,2.438054e+08,4.000000,0.000000,0.000000
50%,2.280712e+07,4.852615e+08,5.000000,0.000000,0.000000
75%,3.867224e+07,7.304523e+08,5.000000,0.000000,0.000000
max,5.307619e+07,9.992091e+08,5.000000,378.000000,383.000000


In [5]:
data.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date'],
      dtype='object')

## Data preprocessing: text data

In [6]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext

In [33]:
! wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

In [38]:
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [39]:
!pip install -q findspark

In [60]:
os.environ["JAVA_HOME"] = "/Applications/jdk-11.0.14.jdk/Contents/Home "
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
os.environ["SPARK_HOME"] = "/Applications/spark-3.2.1-bin-hadoop3.2"

In [58]:
! java -version

No Java runtime present, requesting install.


In [51]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [61]:
text=data['review_headline']+[' ']+data['review_body']

In [51]:
text.head

<bound method NDFrame.head of 0       Great swaddled blankets Loved these swaddle bl...
1       Too cute and really nice These are adorable pa...
2                                   Five Stars Great gift
3       Cute; wash up nicely in dishwasher. These fork...
4       Love it! I wanted something for piece of mind ...
                              ...                        
4995    Great Very sturdy. My toddler loves it. Works ...
4996    Good product. Fits as expected and behaves as ...
4997                                   Five Stars Love jt
4998    Does not work It does not work for my 2 door r...
4999    Been using for 2 months. Doesn't take up too ....
Length: 5000, dtype: object>

AttributeError: module 'sparknlp' has no attribute 'createDataFrame'

No Java runtime present, requesting install.
/usr/local/lib/python3.9/site-packages/pyspark/bin/spark-class: line 96: CMD: bad array subscript
head: illegal line count -- -1


RuntimeError: Java gateway process exited before sending its port number

## Data clean techniques: check missing, NA, null, NaN, empty values 

In [28]:
np.sum(data.isna())

marketplace            0
customer_id            0
review_id              0
product_id             0
product_parent         0
product_title          0
product_category       0
star_rating            0
helpful_votes          2
total_votes            2
vine                   2
verified_purchase      2
review_headline        4
review_body          137
review_date            3
dtype: int64

In [38]:
data['product_id'].nunique()

3797